In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

In [2]:
file_path=r'C:\Users\bhavrang\Documents\Analytics vidhya\GlobalDataChallenge-II\global_data_science_challenge_2_public-master/data/gdsc2_public.csv'
df=pd.read_csv(file_path, sep=';', parse_dates=['timestamp'])
df=df.sort_values(by=['work_item','timestamp'])

In [4]:
#importing userdefined modules to perform relevant tasks on given dataset
import source_func as sf

In [78]:
import imp
imp.reload(sf)

<module 'source_func' from 'C:\\Users\\bhavrang\\Documents\\Analytics vidhya\\GlobalDataChallenge-II\\source_func.py'>

In [79]:
times=sf.compute_work_item_times(df)
times.head()

,work_item,start,end,duration,duration_in_days
0,WI_000001,2015-01-02 14:39:14,2015-01-27 11:36:51,24 days 20:57:37,24.87
1,WI_000002,2015-01-02 15:04:20,2015-01-14 09:46:37,11 days 18:42:17,11.78
2,WI_000003,2015-01-02 15:28:22,2015-02-26 11:50:37,54 days 20:22:15,54.85
3,WI_000004,2015-01-02 15:33:54,2015-01-28 09:11:05,25 days 17:37:11,25.73
4,WI_000005,2015-01-02 16:32:11,2015-02-04 12:57:49,32 days 20:25:38,32.85


In [6]:
ticket_info=sf.tkts_per_day(times)
ticket_info.head()

,date,open_tickets_per_day,closed_tickets_per_day,open_tickets_total,closed_tickets_total,wip_tickets_total
0,2015-01-02,6,0,6,0,6
1,2015-01-03,1,0,7,0,7
2,2015-01-04,2,0,9,0,9
3,2015-01-05,22,0,31,0,31
4,2015-01-06,17,7,48,7,41


In [14]:
sep_date_str = '01.01.2018'
train_times, test_times, closed_times, open_times=sf.split_times(times,sep_date_str)
train_work_items= set(train_times.work_item)
test_work_items= set(test_times.work_item)
closed_work_items= set(closed_times.work_item)
open_work_items= set(open_times.work_item)

In [82]:
cols_to_encode=['work_type', 'work_priority', 'domain','platform','components']
df_start_only = df[df.from_phase == 'Start']
train_df = df_start_only[df_start_only['work_item'].isin(train_work_items)]  
test_df = df_start_only[df_start_only['work_item'].isin(test_work_items)]
closed_df=df_start_only[df_start_only['work_item'].isin(closed_work_items)]
open_df=df_start_only[df_start_only['work_item'].isin(open_work_items)]

In [83]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
# There will be components that only appear in the test set. To deal with this, we change the handle_unknown flag.
train_features = onehotencoder.fit_transform(train_df[cols_to_encode])
test_features = onehotencoder.transform(test_df[cols_to_encode])

In [84]:
closed_features = onehotencoder.fit_transform(closed_df[cols_to_encode])
open_features = onehotencoder.transform(open_df[cols_to_encode])

In [68]:
onehotencoder.get_feature_names()

array(['x0_WT_00001', 'x0_WT_00002', 'x0_WT_00003', 'x1_WP_00001',
       'x1_WP_00002', 'x1_WP_00003', 'x1_WP_00004', 'x1_WP_00005',
       'x2_BD_00001', 'x2_BD_00002', 'x2_BD_00003', 'x2_BD_00004',
       'x2_BD_00005', 'x2_BD_00006', 'x2_BD_00007', 'x2_BD_00008',
       'x2_BD_00009', 'x2_BD_00010', 'x2_BD_00011', 'x3_PL_00001',
       'x3_PL_00002', 'x3_PL_00003', 'x3_PL_00004', 'x3_PL_00005',
       'x3_PL_00006', 'x3_PL_00007', 'x3_PL_00008', 'x3_PL_00009',
       'x3_PL_00010', 'x3_PL_00011', 'x3_PL_00012', 'x3_PL_00013',
       'x3_PL_00014', 'x3_PL_00015', 'x3_PL_00016', 'x3_PL_00017',
       'x3_PL_00018', 'x3_PL_00019'], dtype=object)

In [90]:
#for train and test set with in the closed tkts
x_train=train_features
y_train=train_times
x_test=test_features
y_test=test_times

In [73]:
#for closed and open set 
x_train=closed_features
y_train=closed_times
x_test=open_features
y_test=open_times

In [91]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train.duration_in_days)
y_pred=pd.DataFrame(y_test.work_item)
y_pred['predictions']=lr.predict(x_test)

In [89]:
sf.rmsle(y_test,y_pred)

nan

In [75]:
y_pred.to_csv('Submission_lr.csv',index=False)